In [ ]:
# default_exp core

In [ ]:
#hide
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#hide
!pip install fastcore
from fastcore.all import *
%load_ext autoreload 
%autoreload 2
%cd "/content/drive/MyDrive/Coding/ModelAssistedLabel"

     |████████████████████████████████| 61kB 4.5MB/s 
/content/drive/MyDrive/Coding/ModelAssistedLabel


In [ ]:
from ModelAssistedLabel.core import Generation, Defaults

In [ ]:
# export
from ModelAssistedLabel.core import Trainer
from datetime import datetime

class WeightMaker():
  def __init__(self, resource_dir, name="DEFAULT"):
    self.resource_dir = resource_dir
    self.name = name
    self.g = None
    self.resource_paths = ["test/", "train/", "valid/", "data.yaml"]

  def cleanup():
    for r in self.resource_paths:
      os.system(f"rm -f -r '{r}'")

  def prepare_split(self, out_dir, data_yaml=None, MAX_SIZE=5, verbose=True):
    if data_yaml is None:
      data_yaml = Defaults().data_yaml

    g = Generation(repo=self.resource_dir, out_dir=out_dir, data_yaml=data_yaml)
    g.set_split(MAX_SIZE=MAX_SIZE)

    if verbose:
      def tostr(split):
        return [{k: len(v)} for k,v in split.items()]
      def sumg(split):
        return sum([list(x.values())[0] for x in tostr(split)])
      print("summary: ", tostr(g.split))
      print("checksum:", sumg(g.split))

    self.g = g
  
  def split_and_organize_folders(self):
    assert self.g is not None, "run `prepare_split` first"
    zipped = self.g.process_split(self.name) #create a zip file in the ROOT directory
    local = os.path.basename(zipped)
    os.system(f'unzip "{local}"') #grab data
    #move the contents of the zip file into postion within the ROOT directory
    for content in self.resource_paths:
      os.system(f"mv '{local[:-4]}/{content}' .")
    #remove zip file
    os.system(f"rm -f -r '{local}'")
    #removed the folder that was taken out of the zip
    os.system(f"rm -f -r '{local[:-4]}'")

  def generate_weights(self, epochs, out_dir = ".", train_path = "yolov5/runs/train"):
    self.prepare_split(out_dir)
    self.split_and_organize_folders()

    t = Trainer(self.name)
    ldir = lambda path: set(os.listdir(path))

    before = ldir(train_path)
    t.train(epochs)
    after = ldir(train_path)
    
    test_eq(len(after), len(before)+1) #only should have made one new file
    diff = list(after - before)[0]
    return os.path.join(train_path, diff)

  def tidy_weights(self, out_dir=None):
    if out_dir is None:
      out_dir = repo
    results_path = self.generate_weights()

    while True:
      now = datetime.now()
      out = "{os.path.basename(results_path)}-{now.strftime('%f')}"
      outfolder = os.path.join(out_dir, moved_filename)
      if not os.path.exists(outfolder):
        break
    os.system(f"mv '{results_path}' '{outfolder}'")
    return outfolder

In [ ]:
repo = "/content/drive/MyDrive/Coding/Roboflow Export (841)"
name = "nospaces"
wm = WeightMaker(repo, name)

In [ ]:
wm.prepare_split()

TypeError: ignored

In [ ]:
t.train(1)

In [ ]:
%ls yolov5/runs/train/newname

confusion_matrix.png                               results.png
events.out.tfevents.1615897485.64fc78c672e1.711.0  results.txt
hyp.yaml                                           test_batch0_labels.jpg
labels_correlogram.jpg                             test_batch0_pred.jpg
labels.jpg                                         train_batch0.jpg
opt.yaml                                           weights/


In [ ]:
%ls

00_ultralytics.ipynb   docker-compose.yml  ModelAssistedLabel/  train/
01_split.ipynb         docs/               README.md            valid/
02_train.ipynb         index.ipynb         settings.ini         yolov5/
03_make_weights.ipynb  LICENSE             setup.py
CONTRIBUTING.md        Makefile            _Synch.ipynb
data.yaml              MANIFEST.in         test/
